In [38]:
#1.1 - IMPORT THE NECESSARY LIBRARIES
# !pip install --upgrade pip
# !pip install numpy
# !pip install open3d
# !pip install plotly
# !pip install matplotlib
# !pip install sklearn
import os
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from sklearn.neighbors import NearestNeighbors

In [39]:
#To get the names of all the files to later segment and visualize all the meshes in scaled folder

def get_fileNames(folder_path):
    
    # create an empty list to store the file names
    file_names = []

    # go through each file in the folder
    for filename in os.listdir(folder_path):
        # add the file name to the list
        file_names.append(filename)

    # print the list of file names
    return(file_names)

In [40]:
def estimate_epsilon(dataset):
    # Compute distances to 20th nearest neighbor
    neighbors = NearestNeighbors(n_neighbors=20)
    neighbors_fit = neighbors.fit(dataset)
    distances, indices = neighbors_fit.kneighbors(dataset)

    distances = np.sort(distances, axis=0)
    distances = distances[:,1]

    # Compute curvature
    dx = 1
    dy = np.gradient(distances, dx)
    d2y = np.gradient(dy, dx)
    curvature = np.abs(d2y) / (1 + dy**2)**(3/2)

    # Find point of maximum curvature
    max_curvature_idx = np.argmax(curvature)
    max_curvature_point = (max_curvature_idx, distances[max_curvature_idx])
    
#     print("this is the max curvature point/epsilon:",max_curvature_point[1])
    
# Plot curve and maximum curvature point
#     plt.plot(distances)
#     plt.plot(*max_curvature_point, 'ro')
#     plt.show()
    return(max_curvature_point[1]) #estimated epsilon

In [41]:


def get_best_distance_threshold(point_cloud):
    """
    Calculates the best distance threshold value for a given point cloud.

    Args:
        point_cloud (open3d.geometry.PointCloud): Point cloud to calculate threshold for.

    Returns:
        float: Best distance threshold value.
    """
    distances = point_cloud.compute_nearest_neighbor_distance()
    mean_dist = np.mean(distances)
    std_dist = np.std(distances)
    threshold = mean_dist + 0.5 * std_dist

    return threshold


In [42]:
def refined_ransac(pcd):
    segment_models={}
    segments={}
    max_plane_idx=20
    rest=pcd
    d_threshold= get_best_distance_threshold(pcd)
    for i in range(max_plane_idx):
        colors = plt.get_cmap("tab20")(i)
        segment_models[i], inliers = rest.segment_plane(distance_threshold=d_threshold,ransac_n=3,num_iterations=1000)
        segments[i]=rest.select_by_index(inliers)
        labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
        candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
        best_candidate=int(np.array(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])[0])
        print("the best candidate is: ", best_candidate)
        rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
        segments[i]=segments[i].select_by_index(list(np.where(labels==best_candidate)[0]))
        segments[i].paint_uniform_color(list(colors[:3]))
        print("pass",i+1,"/",max_plane_idx,"done.")
    return rest, segments,max_plane_idx
    

In [43]:
def euclidean_dbscan(rest): 
    dataset = np.array(rest.points)
    epsilon = estimate_epsilon(dataset)
    labels = np.array(rest.cluster_dbscan(eps=epsilon, min_points=5))
    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")

    colors = plt.get_cmap("tab10")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    rest.colors = o3d.utility.Vector3dVector(colors[:, :3])
    print("\n\n")
    return(rest)

In [44]:
def draw_plotly(rest,segments):
    #o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])
    points = np.asarray(rest.points)

    fig = go.Figure()

    if rest.colors != None:
        colors = np.asarray(rest.colors)
    else:
        colors = np.ones(len(np.asarray(rest.points)))

    fig.add_traces(go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=1, color=colors)))
    for i in range(len(segments)):
        pc=np.asarray(segments[i].points)
        fig.add_traces(go.Scatter3d(x=pc[:,0], y=pc[:,1], z=pc[:,2], 
            mode='markers',
            marker=dict(size=1, color=np.asarray(segments[i].colors))))
    fig.update_layout(dict(scene = dict(aspectmode='data')))
    fig.show()

In [45]:
#Getting the names of all files in the Archive folder
fileNames = get_fileNames("Archive")

#making a mesh of the stl file; please change the current file index number to see the segmentation of other 
#stl meshes
current_file = 3
mesh = o3d.io.read_triangle_mesh('Archive'+'/'+fileNames[current_file])  

pcd = mesh.sample_points_uniformly(number_of_points=2000000)    


In [46]:
#applying ransac on the point cloud and dbscan on each segment and colorizing them
rest,segments,max_plane_idx = refined_ransac(pcd)

#applying ransac on the rest of the points (outliers)
rest = euclidean_dbscan(rest)

the best candidate is:  0
pass 1 / 20 done.
the best candidate is:  0
pass 2 / 20 done.
the best candidate is:  0
pass 3 / 20 done.
the best candidate is:  0
pass 4 / 20 done.
the best candidate is:  0
pass 5 / 20 done.
the best candidate is:  0
pass 6 / 20 done.
the best candidate is:  0
pass 7 / 20 done.
the best candidate is:  0
pass 8 / 20 done.
the best candidate is:  0
pass 9 / 20 done.
the best candidate is:  0
pass 10 / 20 done.
the best candidate is:  9
pass 11 / 20 done.
the best candidate is:  8
pass 12 / 20 done.
the best candidate is:  0
pass 13 / 20 done.
the best candidate is:  0
pass 14 / 20 done.
the best candidate is:  1
pass 15 / 20 done.
the best candidate is:  0
pass 16 / 20 done.
the best candidate is:  0
pass 17 / 20 done.
the best candidate is:  1
pass 18 / 20 done.
the best candidate is:  14
pass 19 / 20 done.
the best candidate is:  0
pass 20 / 20 done.
point cloud has 47 clusters





In [47]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])
# draw_plotly(rest,segments)